# MCTS + Whittle Indices Experiments

Analyze the performance of various algorithms to solve the joint matching + activity task, when the number of volunteers is large and structured

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
import secrets

In [3]:
from rmab.simulator import RMABSimulator, run_heterogenous_policy, get_discounted_reward
from rmab.omniscient_policies import *
from rmab.fr_dynamics import get_all_transitions
from rmab.compute_whittle import arm_compute_whittle_multi_prob
from rmab.mcts_policies import *
from rmab.utils import get_save_path, delete_duplicate_results, create_prob_distro
import resource

/usr0/home/naveenr/miniconda3/envs/food/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.cuda.set_per_process_memory_fraction(0.5)
torch.set_num_threads(1)
resource.setrlimit(resource.RLIMIT_AS, (30 * 1024 * 1024 * 1024, -1))

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [9]:
if is_jupyter: 
    seed        = 43
    n_arms      = 4
    volunteers_per_arm = 1
    budget      = 2
    discount    = 0.9
    alpha       = 3 
    n_episodes  = 20
    episode_len = 20 
    n_epochs    = 1
    save_with_date = False 
    TIME_PER_RUN = 0.01 * 1000
    lamb = 0
    prob_distro = 'uniform'
    reward_type = "max"
    reward_parameters = {'universe_size': 20, 'arm_set_low': 15, 'arm_set_high': 20}
    policy_lr=5e-3
    value_lr=1e-4
    train_iterations = 30
    test_iterations = 30
    out_folder = 'mcts_exploration/mcts_shapley'
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--n_arms',         '-N', help='num beneficiaries (arms)', type=int, default=2)
    parser.add_argument('--volunteers_per_arm',         '-V', help='volunteers per arm', type=int, default=5)
    parser.add_argument('--episode_len',    '-H', help='episode length', type=int, default=20)
    parser.add_argument('--n_episodes',     '-T', help='num episodes', type=int, default=1)
    parser.add_argument('--budget',         '-B', help='budget', type=int, default=3)
    parser.add_argument('--n_epochs',       '-E', help='number of epochs (num_repeats)', type=int, default=3)
    parser.add_argument('--discount',       '-d', help='discount factor', type=float, default=0.9)
    parser.add_argument('--alpha',          '-a', help='alpha: for conf radius', type=float, default=3)
    parser.add_argument('--lamb',          '-l', help='lambda for matching-engagement tradeoff', type=float, default=0.5)
    parser.add_argument('--universe_size', help='For set cover, total num unvierse elems', type=int, default=10)
    parser.add_argument('--arm_set_low', help='Least size of arm set, for set cover', type=int, default=3)
    parser.add_argument('--arm_set_high', help='Largest size of arm set, for set cover', type=int, default=6)
    parser.add_argument('--reward_type',          '-r', help='Which type of custom reward', type=str, default='set_cover')
    parser.add_argument('--seed',           '-s', help='random seed', type=int, default=42)
    parser.add_argument('--prob_distro',           '-p', help='which prob distro [uniform,uniform_small,uniform_large,normal]', type=str, default='uniform')
    parser.add_argument('--time_per_run',      '-t', help='time per MCTS run', type=float, default=.01*1000)
    parser.add_argument('--policy_lr', help='Learning Rate Policy', type=float, default=5e-3)
    parser.add_argument('--value_lr', help='Learning Rate Value', type=float, default=1e-4)
    parser.add_argument('--train_iterations', help='Number of MCTS train iterations', type=int, default=30)
    parser.add_argument('--test_iterations', help='Number of MCTS test iterations', type=int, default=30)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='mcts_exploration/mcts_shapley')

    parser.add_argument('--use_date', action='store_true')

    args = parser.parse_args()

    n_arms      = args.n_arms
    volunteers_per_arm = args.volunteers_per_arm
    budget      = args.budget
    discount    = args.discount
    alpha       = args.alpha 
    seed        = args.seed
    n_episodes  = args.n_episodes
    episode_len = args.episode_len
    n_epochs    = args.n_epochs
    lamb = args.lamb
    save_with_date = args.use_date
    TIME_PER_RUN = args.time_per_run
    prob_distro = args.prob_distro
    policy_lr = args.policy_lr 
    value_lr = args.value_lr 
    out_folder = args.out_folder
    train_iterations = args.train_iterations 
    test_iterations = args.test_iterations 
    reward_type = args.reward_type
    reward_parameters = {'universe_size': args.universe_size,
                        'arm_set_low': args.arm_set_low, 
                        'arm_set_high': args.arm_set_high}

save_name = secrets.token_hex(4)  

In [10]:
n_states = 2
n_actions = 2

In [11]:
all_population_size = 100 
all_transitions = get_all_transitions(all_population_size)
all_transitions[all_transitions > 0.5] = 1 # np.random.random(all_transitions[all_transitions > 0.5].shape)*(1-1) + 1
all_transitions[all_transitions <= 0.5] = 0

In [12]:
for i in range(all_population_size):
    a = np.random.random()
    b = a+np.random.random()*(1-a)
    c = a+np.random.random()*(1-a)
    d = max(b,c)
    d += np.random.random()*(1-d)
    all_transitions[i] = np.array([[[1-a,a],[1-b,b]],[[1-c,c],[1-d,d]]])

In [16]:
def create_environment(seed):
    random.seed(seed)
    np.random.seed(seed)

    all_features = np.arange(all_population_size)
    if reward_type == "set_cover":
        match_probabilities = [set([random.randint(0,reward_parameters['universe_size']) for _ in range(random.randint(reward_parameters['arm_set_low'],reward_parameters['arm_set_high']))]) for i in range(all_population_size*volunteers_per_arm)]
    else:
        match_probabilities = np.array(create_prob_distro(prob_distro,all_population_size*volunteers_per_arm))

    simulator = RMABSimulator(all_population_size, all_features, all_transitions,
                n_arms, volunteers_per_arm, episode_len, n_epochs, n_episodes, budget, discount,number_states=n_states, reward_style='custom',match_probability_list=match_probabilities,TIME_PER_RUN=TIME_PER_RUN)
    simulator.reward_type = reward_type 
    simulator.reward_parameters = reward_parameters 

    return simulator 

In [17]:
def run_multi_seed(seed_list,policy,is_mcts=False,per_epoch_function=None,train_iterations=0,test_iterations=0,test_length=20):
    memories = []
    scores = {
        'reward': [],
        'time': [], 
        'match': [], 
        'active_rate': [],
    }

    for seed in seed_list:
        simulator = create_environment(seed)

        if is_mcts:
            simulator.mcts_train_iterations = train_iterations
            simulator.mcts_test_iterations = test_iterations
            simulator.policy_lr = policy_lr
            simulator.value_lr = value_lr

        if is_mcts:
            match, active_rate, memory = run_heterogenous_policy(simulator, n_episodes, n_epochs, discount,policy,seed,lamb=lamb,should_train=True,test_T=test_length,get_memory=True,per_epoch_function=per_epoch_function)
        else:
            match, active_rate = run_heterogenous_policy(simulator, n_episodes, n_epochs, discount,policy,seed,lamb=lamb,should_train=True,test_T=test_length,per_epoch_function=per_epoch_function)
        time_whittle = simulator.time_taken
        discounted_reward = get_discounted_reward(match,active_rate,discount,lamb)
        scores['reward'].append(discounted_reward)
        scores['time'].append(time_whittle)
        scores['match'].append(np.mean(match))
        scores['active_rate'].append(np.mean(active_rate))
        if is_mcts:
            memories.append(memory)

    return scores, memories, simulator

In [18]:
results = {}
results['parameters'] = {'seed'      : seed,
        'n_arms'    : n_arms,
        'volunteers_per_arm': volunteers_per_arm, 
        'budget'    : budget,
        'discount'  : discount, 
        'alpha'     : alpha, 
        'n_episodes': n_episodes, 
        'episode_len': episode_len, 
        'n_epochs'  : n_epochs, 
        'lamb': lamb,
        'time_per_run': TIME_PER_RUN, 
        'prob_distro': prob_distro, 
        'policy_lr': policy_lr, 
        'value_lr': value_lr, 
        'reward_type': reward_type, 
        'universe_size': reward_parameters['universe_size'],
        'arm_set_low': reward_parameters['arm_set_low'], 
        'arm_set_high': reward_parameters['arm_set_high'],
        } 

## Index Policies

In [26]:
all_diffs = []
for seed in range(40,50):
    seed_list = [seed]
    policy = mcts_shapley_policy
    name = "mcts_shapley"

    rewards, memory, simulator = run_multi_seed(seed_list,policy,is_mcts=True,test_iterations=400,test_length=n_episodes*episode_len)
    results['{}_reward'.format(name)] = rewards['reward']
    results['{}_match'.format(name)] =  rewards['match'] 
    results['{}_active'.format(name)] = rewards['active_rate']
    results['{}_time'.format(name)] =  rewards['time']
    rew_whittle = np.mean(rewards['reward'])
    policy = q_iteration_policy
    per_epoch_function = q_iteration_custom_epoch()
    name = "optimal"

    print("Running optimal")

    rewards, memory, simulator = run_multi_seed(seed_list,policy,per_epoch_function=per_epoch_function,test_length=n_episodes*episode_len)
    results['{}_reward'.format(name)] = rewards['reward']
    results['{}_match'.format(name)] =  rewards['match'] 
    results['{}_active'.format(name)] = rewards['active_rate']
    results['{}_time'.format(name)] =  rewards['time']
    rew_q = np.mean(rewards['reward'])

    diff = rew_whittle/rew_q
    all_diffs.append(diff)

cohort [69 63 50 49]
instance 0, ep 1
instance 0, ep 2
instance 0, ep 3
instance 0, ep 4
instance 0, ep 5
instance 0, ep 6
instance 0, ep 7
instance 0, ep 8
instance 0, ep 9
instance 0, ep 10
instance 0, ep 11
instance 0, ep 12
instance 0, ep 13
instance 0, ep 14
instance 0, ep 15
instance 0, ep 16
instance 0, ep 17
instance 0, ep 18
instance 0, ep 19
Took 30.76919412612915 time for inference and 0.5564303398132324 time for training
Running optimal
cohort [69 63 50 49]
instance 0, ep 1
instance 0, ep 2
instance 0, ep 3
instance 0, ep 4
instance 0, ep 5
instance 0, ep 6
instance 0, ep 7
instance 0, ep 8
instance 0, ep 9
instance 0, ep 10
instance 0, ep 11
instance 0, ep 12
instance 0, ep 13
instance 0, ep 14
instance 0, ep 15
instance 0, ep 16
instance 0, ep 17
instance 0, ep 18
instance 0, ep 19
Took 0.0386049747467041 time for inference and 2.517144203186035 time for training
cohort [69 46 10 27]
instance 0, ep 1
instance 0, ep 2
instance 0, ep 3
instance 0, ep 4
instance 0, ep 5
inst

In [27]:
np.mean(all_diffs), np.max(all_diffs), np.min(all_diffs)

(0.9997954694702627, 1.0017727007741208, 0.9972490160558649)

In [204]:
simulator.transitions[:,0,1,1]

array([0.25      , 0.15      , 0.86886303])

acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [67]
instance 0, ep 1
instance 0, ep 2
instance 0, ep 3
instance 0, ep 4
Took 0.2290353775024414 time for inference and 0.23792767524719238 time for training
7.987503284153403


acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [67]
instance 0, ep 1
instance 0, ep 2
instance 0, ep 3
instance 0, ep 4
Took 0.009995460510253906 time for inference and 2.818249464035034 time for training
7.987503284153403


In [175]:
pair_amounts = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        pair_amounts[i,j] = len(simulator.match_probability_list[simulator.agent_idx][i].union(simulator.match_probability_list[simulator.agent_idx][j]))
a = [len(i) for i in simulator.match_probability_list[simulator.agent_idx]]
a = [(i,a[i]) for i in range(len(a))]
a = sorted(a,key = lambda k: k[1],reverse=True)
max_pair = 1
while max_pair < len(a)  and a[1][1] == a[max_pair][1]:
    max_pair += 1
max_pair -= 1

found_max_pair = 0
for i in range(max_pair+1):
    for j in range(max_pair+1):
        found_max_pair = max(pair_amounts[a[i][0],a[j][0]],found_max_pair)
found_max_pair/np.max(pair_amounts)

0.8235294117647058

In [151]:
policy = shapley_whittle_custom_policy 
name = "shapley_whittle_custom"

rewards, memory, simulator = run_multi_seed(seed_list,policy,test_length=20)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
print(np.mean(rewards['reward']))

cohort [79]
instance 0, ep 1
instance 0, ep 2
instance 0, ep 3
instance 0, ep 4
Took 0.0029036998748779297 time for inference and 0.4919767379760742 time for training
58.463


In [152]:
policy = mcts_shapley_policy
name = "mcts_shapley"

rewards, memory, simulator = run_multi_seed(seed_list,policy,is_mcts=True,test_iterations=400,test_length=20)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
np.mean(rewards['reward'])

cohort [79]


TypeError: get_attributions() got an unexpected keyword argument 'attribution_method'

In [47]:
policy = mcts_shapley_attributions_policy
name = "mcts_shapley_attribution"

rewards, memory, simulator = run_multi_seed(seed_list,policy,is_mcts=True,test_iterations=400,test_length=20)
results['{}_reward'.format(name)] = rewards['reward']
results['{}_match'.format(name)] =  rewards['match'] 
results['{}_active'.format(name)] = rewards['active_rate']
results['{}_time'.format(name)] =  rewards['time']
np.mean(rewards['reward'])

acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [20]
Took 2.373187303543091 time for inference and 0.4436161518096924 time for training
acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [42]
Took 2.027259588241577 time for inference and 0.22554779052734375 time for training
acting should always be good! 0.108 < 0.183
good start state should always be good! 0.380 < 0.508
good start state should always be good! 0.506 < 0.760
cohort [0]
Took 1.9015212059020996 time for inference and 0.425368070602417 time for training


28.966393687307782

## Write Data

In [ ]:
save_path = get_save_path(out_folder,save_name,seed,use_date=save_with_date)

In [ ]:
delete_duplicate_results(out_folder,"",results)

In [ ]:
json.dump(results,open('../../results/'+save_path,'w'))